In [1]:
import numpy as np
import time
import sys
import os
import random
from skimage import io
import pandas as pd
from matplotlib import pyplot as plt
from shutil import copyfile
import cv2
import tensorflow as tf

In [2]:
datapath = 'N:\Study\Object Detection'
#images_boxable = 'train-images-boxable.csv'
#annotations_bbox = 'train-annotations-bbox.csv'
#class_descriptions = 'class-descriptions.csv'

In [3]:
images_boxable = pd.read_csv(os.path.join(datapath, 'train-images-boxable.csv'))
images_boxable.head()

,image_name,image_url
0,e39871fd9fd74f55.jpg,https://requestor-proxy.figure-eight.com/figur...
1,f18b91585c4d3f3e.jpg,https://requestor-proxy.figure-eight.com/figur...
2,ede6e66b2fb59aab.jpg,https://requestor-proxy.figure-eight.com/figur...
3,ed600d57fcee4f94.jpg,https://requestor-proxy.figure-eight.com/figur...
4,ff47e649b23f446d.jpg,https://requestor-proxy.figure-eight.com/figur...


In [4]:
annotations_bbox = pd.read_csv(os.path.join(datapath, 'train-annotations-bbox.csv'))

In [6]:
class_descriptions = pd.read_csv(os.path.join(datapath,'class-descriptions.csv'))

In [7]:
# Find the label_name for 'Person', 'Mobile Phone' and 'Car' classes
person = class_descriptions[class_descriptions['class']=='Person']
food = class_descriptions[class_descriptions['class']=='Food']
phone = class_descriptions[class_descriptions['class']=='Mobile phone']
boat = class_descriptions[class_descriptions['class']=='Boat']
car = class_descriptions[class_descriptions['class']=='Car']
motorcycle = class_descriptions[class_descriptions['class']=='Motorcycle']
fruit = class_descriptions[class_descriptions['class']=='Fruit']
flower = class_descriptions[class_descriptions['class']=='Flower']

label_person = person['name'].values[0]
label_food = food['name'].values[0]
label_phone = phone['name'].values[0]
label_boat = boat['name'].values[0]
label_car = car['name'].values[0]
label_motorcycle = motorcycle['name'].values[0]
label_fruit = fruit['name'].values[0]
label_flower = flower['name'].values[0]

In [9]:
person_bbox = annotations_bbox[annotations_bbox['LabelName']==label_person]
food_bbox = annotations_bbox[annotations_bbox['LabelName']==label_food]
phone_bbox = annotations_bbox[annotations_bbox['LabelName']==label_phone]
boat_bbox = annotations_bbox[annotations_bbox['LabelName']==label_boat]
car_bbox = annotations_bbox[annotations_bbox['LabelName']==label_car]
motorcycle_bbox = annotations_bbox[annotations_bbox['LabelName']==label_motorcycle]
fruit_bbox = annotations_bbox[annotations_bbox['LabelName']==label_fruit]
flower_bbox = annotations_bbox[annotations_bbox['LabelName']==label_flower]

In [10]:
print('# of persons: %d' %(len(person_bbox)))
print('# of food: %d' %(len(food_bbox)))
print('# of phones: %d' %(len(phone_bbox)))
print('# of boat: %d' %(len(boat_bbox)))
print('# of cars: %d' %(len(car_bbox)))
print('# of motorcycles: %d' %(len(motorcycle_bbox)))
print('# of fruit: %d' %(len(fruit_bbox)))
print('# of flower: %d' %(len(flower_bbox)))

# of persons: 1034721
# of food: 88422
# of phones: 6365
# of boat: 79113
# of cars: 248075
# of motorcycles: 13382
# of fruit: 26236
# of flower: 345296


In [11]:
person_img_id = person_bbox['ImageID']
phone_img_id = phone_bbox['ImageID']
car_img_id = car_bbox['ImageID']
motorcycle_img_id = motorcycle_bbox['ImageID']
boat_img_id = boat_bbox['ImageID']
fruit_img_id = fruit_bbox['ImageID']
flower_img_id = flower_bbox['ImageID']
food_img_id = food_bbox['ImageID']

In [12]:
flower_img_id = np.unique(flower_img_id)
phone_img_id = np.unique(phone_img_id)
motorcycle_img_id = np.unique(motorcycle_img_id)
person_img_id = np.unique(person_img_id)
boat_img_id = np.unique(boat_img_id)
car_img_id = np.unique(car_img_id)
fruit_img_id = np.unique(fruit_img_id)
food_img_id = np.unique(food_img_id)

In [13]:
print('%d images containing persons' % (len(person_img_id)))
print('%d images containing phones' % (len(phone_img_id)))
print('%d images containing cars' % (len(car_img_id)))
print('%d images containing motorcycle' % (len(motorcycle_img_id)))
print('%d images containing boat' % (len(boat_img_id)))
print('%d images containing food' % (len(food_img_id)))
print('%d images containing fruit' % (len(fruit_img_id)))
print('%d images containing flower' % (len(flower_img_id)))

248384 images containing persons
4312 images containing phones
89465 images containing cars
6944 images containing motorcycle
25478 images containing boat
25712 images containing food
5283 images containing fruit
62716 images containing flower


In [14]:
#rabdomly shuffling img_ids
person_img_cpy = person_img_id.copy()
random.seed(1)
random.shuffle(person_img_cpy)

phone_img_cpy = phone_img_id.copy()
random.seed(1)
random.shuffle(phone_img_cpy)

car_img_cpy = car_img_id.copy()
random.seed(1)
random.shuffle(car_img_cpy)

motorcycle_img_cpy = motorcycle_img_id.copy()
random.seed(1)
random.shuffle(motorcycle_img_cpy)

fruit_img_cpy = fruit_img_id.copy()
random.seed(1)
random.shuffle(fruit_img_cpy)

flower_img_cpy = flower_img_id.copy()
random.seed(1)
random.shuffle(flower_img_cpy)

boat_img_cpy = boat_img_id.copy()
random.seed(1)
random.shuffle(boat_img_cpy)

food_img_cpy = food_img_id.copy()
random.seed(1)
random.shuffle(food_img_cpy)

#selecting 1000 img_ids
n = 1000
person_img_filt = person_img_cpy[:n]
phone_img_filt = phone_img_cpy[:n]
car_img_filt = car_img_cpy[:n]
motorcycle_img_filt = motorcycle_img_cpy[:n]
food_img_filt = food_img_cpy[:n]
boat_img_filt = boat_img_cpy[:n]
fruit_img_filt = fruit_img_cpy[:n]
flower_img_filt = flower_img_cpy[:n]

In [56]:
person_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in person_img_filt]
phone_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in phone_img_filt]
car_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in car_img_filt]
motorcycle_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in motorcycle_img_filt]
food_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in food_img_filt]
boat_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in boat_img_filt]
fruit_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in fruit_img_filt]
flower_img_url = [images_boxable[images_boxable['image_name']==name+'.jpg'] for name in flower_img_filt]

In [58]:
person_df = pd.DataFrame()
phone_df = pd.DataFrame()
car_df = pd.DataFrame()
motorcycle_df = pd.DataFrame()
food_df = pd.DataFrame()
boat_df = pd.DataFrame()
fruit_df = pd.DataFrame()
flower_df = pd.DataFrame()

for i in range(len(person_img_url)):
    person_df = person_df.append(person_img_url[i], ignore_index = True)
    phone_df = phone_df.append(phone_img_url[i], ignore_index = True)
    car_df = car_df.append(car_img_url[i], ignore_index = True)
    motorcycle_df = motorcycle_df.append(motorcycle_img_url[i], ignore_index = True)
    food_df = food_df.append(food_img_url[i], ignore_index = True)
    boat_df = boat_df.append(boat_img_url[i], ignore_index = True)
    fruit_df = fruit_df.append(fruit_img_url[i], ignore_index = True)
    flower_df = flower_df.append(flower_img_url[i], ignore_index = True)

person_df.to_csv(os.path.join(datapath, 'filt_person_img_url.csv'))
phone_df.to_csv(os.path.join(datapath, 'filt_phone_img_url.csv'))
car_df.to_csv(os.path.join(datapath, 'filt_car_img_url.csv'))
motorcycle_df.to_csv(os.path.join(datapath, 'filt_motorcycle_img_url.csv'))
food_df.to_csv(os.path.join(datapath, 'filt_food_img_url.csv'))
boat_df.to_csv(os.path.join(datapath, 'filt_boat_img_url.csv'))
fruit_df.to_csv(os.path.join(datapath, 'filt_fruit_img_url.csv'))
flower_df.to_csv(os.path.join(datapath, 'filt_flower_img_url.csv'))

In [62]:
person_img_url = [url['image_url'].values[0] for url in person_img_url]
phone_img_url = [url['image_url'].values[0] for url in phone_img_url]
car_img_url = [url['image_url'].values[0] for url in car_img_url]
motorcycle_img_url = [url['image_url'].values[0] for url in motorcycle_img_url]
boat_img_url = [url['image_url'].values[0] for url in boat_img_url]
food_img_url = [url['image_url'].values[0] for url in food_img_url]
fruit_img_url = [url['image_url'].values[0] for url in fruit_img_url]
flower_img_url = [url['image_url'].values[0] for url in flower_img_url]

urls = [person_img_url, phone_img_url, car_img_url, motorcycle_img_url, boat_img_url, food_img_url, fruit_img_url, flower_img_url]

TypeError: string indices must be integers

In [63]:
person_df= pd.read_csv(os.path.join(datapath, 'filt_person_img_url.csv'))
phone_df= pd.read_csv(os.path.join(datapath, 'filt_phone_img_url.csv'))
car_df= pd.read_csv(os.path.join(datapath, 'filt_car_img_url.csv'))
motorcycle_df= pd.read_csv(os.path.join(datapath, 'filt_motorcycle_img_url.csv'))
food_df= pd.read_csv(os.path.join(datapath, 'filt_food_img_url.csv'))
boat_df= pd.read_csv(os.path.join(datapath, 'filt_boat_img_url.csv'))
fruit_df= pd.read_csv(os.path.join(datapath, 'filt_fruit_img_url.csv'))
flower_df= pd.read_csv(os.path.join(datapath, 'filt_flower_img_url.csv'))

person_img_url = person_df['image_url'].values
phone_img_url = phone_df['image_url'].values
car_img_url = car_df['image_url'].values
motorcycle_img_url = motorcycle_df['image_url'].values
food_img_url = food_df['image_url'].values
boat_img_url = boat_df['image_url'].values
fruit_img_url = fruit_df['image_url'].values
flower_img_url = flower_df['image_url'].values

urls = [person_img_url, phone_img_url, car_img_url, motorcycle_img_url, boat_img_url, food_img_url, fruit_img_url, flower_img_url]

In [64]:
saved_dirs = [os.path.join(datapath,'Person'),os.path.join(datapath,'Mobile phone'),os.path.join(datapath,'Car'),os.path.join(datapath,'Motorcycle'),os.path.join(datapath,'Boat'),os.path.join(datapath,'Food'),os.path.join(datapath,'Fruit'),os.path.join(datapath,'Flower')]

In [65]:
classes = ['Person', 'Mobile phone', 'Car', 'Motorcycle', 'Boat', 'Food', 'Fruit', 'Flower']

In [79]:
import http.client

In [70]:
print(classes)

['Person', 'Mobile phone', 'Car', 'Motorcycle', 'Boat', 'Food', 'Fruit', 'Flower']


In [87]:
for i in range(562,len(urls[1])):
        try:
            img = io.imread(urls[1][i])
            if (img.shape[0]==2):
                img = img[0]
        except (http.client.IncompleteRead) as e:
            img = e.partial
        saved_path = os.path.join('N:\Study\Object Detection\Mobile Phone', urls[1][i][-20:])
        io.imsave(saved_path, img)

In [88]:
for i in range(81,len(urls[2])):
        try:
            img = io.imread(urls[2][i])
            if (img.shape[0]==2):
                img = img[0]
        except (http.client.IncompleteRead) as e:
            img = e.partial
        saved_path = os.path.join('N:\Study\Object Detection\Car', urls[2][i][-20:])
        io.imsave(saved_path, img)

In [89]:
for i in range(3,len(classes)):
    # Create the directory
    os.mkdir(saved_dirs[i])
    dir = saved_dirs[i]
    for url in urls[i]:
        # print(url)
        try:
            img = io.imread(url)
            if (img.shape[0]==2):
                img = img[0]
        except (http.client.IncompleteRead) as e:
            img = e.partial
        saved_path = os.path.join(dir, url[-20:])
        io.imsave(saved_path, img)

In [ ]:
train_path = os.path.join(datapath, 'train')
os.mkdir(train_path)
test_path = os.path.join(datapath, 'test')
os.mkdir(test_path)

In [92]:
for i in range(len(classes)):    
    tot_img = os.listdir(os.path.join(datapath, classes[i]))
    tot_img = [f for f in tot_img if not f.startswith('.')]
    random.seed(1)
    random.shuffle(tot_img)
    
    train_img = tot_img[:800]
    test_img = tot_img[800:]
    
    # Copy each classes' images to train directory
    for j in range(len(train_img)):
        old_path = os.path.join(os.path.join(datapath, classes[i]), train_img[j])
        new_path = os.path.join(train_path, train_img[j])
        copyfile(old_path, new_path)
    
    # Copy each classes' images to test directory
    for j in range(len(test_img)):
        old_path = os.path.join(os.path.join(datapath, classes[i]), test_img[j])
        new_path = os.path.join(test_path, test_img[j])
        copyfile(old_path, new_path)

In [93]:
print('Total images in train: ', len(os.listdir(train_path))) # subtract one because there is one hidden file named '.DS_Store'
print('Total images in train: ', len(os.listdir(test_path)))

Total images in train:  6395
Total images in train:  1598


In [98]:
labels = [label_person, label_phone, label_car, label_motorcycle, label_boat, label_food, label_fruit, label_flower]

In [99]:
print(labels)

['/m/01g317', '/m/050k8', '/m/0k4j', '/m/04_sv', '/m/019jd', '/m/02wbm', '/m/02xwb', '/m/0c9ph5']


In [ ]:
train_df, test_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])

In [100]:
# Find boxes in each image and put them in a dataframe
train_img = os.listdir(train_path)
train_img = [name for name in train_img if not name.startswith('.')]

for i in range(len(train_img)):
    sys.stdout.write('Parse train_img ' + str(i) + '; Number of boxes: ' + str(len(train_df)) + '\r')
    sys.stdout.flush()
    file_name = train_img[i]
    file_id = file_name[0:16]
    temp_df = annotations_bbox[annotations_bbox['ImageID']==file_id]
    for index, row in temp_df.iterrows():
        label_Name = row['LabelName']
        for i in range(len(labels)):
            if label_Name == labels[i]:
                train_df = train_df.append({'FileName': file_name, 
                                            'XMin': row['XMin'], 
                                            'XMax': row['XMax'], 
                                            'YMin': row['YMin'], 
                                            'YMax': row['YMax'], 
                                            'ClassName': classes[i]}, 
                                           ignore_index=True)

In [101]:
test_df = pd.DataFrame(columns=['FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName'])
# Find boxes in each image and put them in a dataframe
test_img = os.listdir(test_path)
test_img = [name for name in test_img if not name.startswith('.')]

for i in range(len(test_img)):
    sys.stdout.write('Parse test_imgs ' + str(i) + '; Number of boxes: ' + str(len(test_df)) + '\r')
    sys.stdout.flush()
    file_name = test_img[i]
    file_id = file_name[0:16]
    temp_df = annotations_bbox[annotations_bbox['ImageID']==file_id]
    for index, row in temp_df.iterrows():
        label_Name = row['LabelName']
        for i in range(len(labels)):
            if label_Name == labels[i]:
                test_df = test_df.append({'FileName': file_name, 
                                            'XMin': row['XMin'], 
                                            'XMax': row['XMax'], 
                                            'YMin': row['YMin'], 
                                            'YMax': row['YMax'], 
                                            'ClassName': classes[i]}, 
                                           ignore_index=True)

In [102]:
train_df.to_csv(os.path.join(datapath, 'train.csv'))
test_df.to_csv(os.path.join(datapath, 'test.csv'))

In [103]:
# print(train_df.head())
train_df = pd.read_csv(os.path.join(datapath, 'train.csv'))
#file_path = os.path.join(train_path,'train')
#os.mkdir(file_path)

# For training
f= open(datapath + "/annotation.txt","w+")
for idx, row in train_df.iterrows():
    sys.stdout.write(str(idx) + '\r')
    sys.stdout.flush()
    img = cv2.imread((datapath + '/train/' + row['FileName']))
    height, width = img.shape[:2]
    x1 = int(row['XMin'] * width)
    x2 = int(row['XMax'] * width)
    y1 = int(row['YMin'] * height)
    y2 = int(row['YMax'] * height)
    
    fileName = os.path.join(train_path, row['FileName'])
    className = row['ClassName']
    f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')
f.close()

In [104]:
#print(test_df.head())
test_df = pd.read_csv(os.path.join(datapath, 'test.csv'))

# For test
f= open(datapath + "/test_annotation.txt","w+")
for idx, row in test_df.iterrows():
    sys.stdout.write(str(idx) + '\r')
    sys.stdout.flush()
    img = cv2.imread((datapath + '/test/' + row['FileName']))
    height, width = img.shape[:2]
    x1 = int(row['XMin'] * width)
    x2 = int(row['XMax'] * width)
    y1 = int(row['YMin'] * height)
    y2 = int(row['YMax'] * height)
    
    fileName = os.path.join(test_path, row['FileName'])
    className = row['ClassName']
    f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')
f.close()